In [2]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageFilter
import cv2 as cv
import numpy as np
import os
from skimage.feature import hog
window = tk.Tk()
window.title("Đồ án")
window.geometry('1200x800')



folder_path = ""
blur_folder = "blur_folder"
giananh_folder = "giananh_folder"
coanh_folder = "coanh_folder"
trichbien_folder = "trichbien_folder"
lamday_folder = "lamday_folder"
laynguong_folder = "laynguong_folder"
ruttrich_folder_hog = "ruttrich_folder_hog"
ruttrich_folder_sift = "ruttrich_folder_sift"

lamnet_folder = "lamnet_folder"
lamsang_folder = "lamsang_folder"
nangcaocl_folder = "nangcaocl_folder"
phanvung_folder = "phanvung_folder"
image_list = []
current_image_index = 0

placeholder_image = Image.open("image/jack_1.jpg")
placeholder_image = placeholder_image.resize((200, 200))
placeholder_photo = ImageTk.PhotoImage(placeholder_image)

image_label = tk.Label(window, image=placeholder_photo)
image_label.grid(column=2, row=1)
image_label.place(x=50, y=70)

image_label = tk.Label(window, image=placeholder_photo)
image_label.grid(column=2, row=1)
image_label.place(x=50, y=70)

def button_clicked():
    # Apply blur effect to the image and update the image_label
    image = placeholder_image.filter(ImageFilter.BLUR)  # Apply blur filter (modify with your desired processing)
    image = image.resize((200, 200))
    blurred_photo = ImageTk.PhotoImage(image)
    image_label.configure(image=blurred_photo)
    image_label.image = blurred_photo

def lamsanganh():
    # Apply brightness and contrast adjustment to the image and update the image_label
    brightness = 10
    contrast = 2.3
    adjusted_image = placeholder_image.point(lambda p: p * contrast + brightness)
    adjusted_image = adjusted_image.resize((200, 200))
    adjusted_photo = ImageTk.PhotoImage(adjusted_image)
    image_label.configure(image=adjusted_photo)
    image_label.image = adjusted_photo

def lamsacnet():
    # Apply sharpening filter to the image and update the image_label
    image_cv = cv.cvtColor(np.array(placeholder_image), cv.COLOR_RGB2BGR)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=np.float32)
    sharpened_image = cv.filter2D(image_cv, -1, kernel)
    sharpened_image = Image.fromarray(cv.cvtColor(sharpened_image, cv.COLOR_BGR2RGB))
    sharpened_image = sharpened_image.resize((200, 200))
    sharpened_photo = ImageTk.PhotoImage(sharpened_image)
    image_label.configure(image=sharpened_photo)
    image_label.image = sharpened_photo

def gian_anh():
    # Convert the image to OpenCV format
    image_cv = cv.cvtColor(np.array(placeholder_image), cv.COLOR_RGB2BGR)
    
    # Create a structuring element for dilation
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (5, 5))
    
    # Apply dilation to the image
    dilated_image = cv.dilate(image_cv, kernel, iterations=1)
    
    # Convert the image back to PIL format
    dilated_image = Image.fromarray(cv.cvtColor(dilated_image, cv.COLOR_BGR2RGB))
    dilated_image = dilated_image.resize((200, 200))
    dilated_photo = ImageTk.PhotoImage(dilated_image)
    
    # Update the image_label
    image_label.configure(image=dilated_photo)
    image_label.image = dilated_photo


def image_segmentation():
    # Perform image segmentation
    num_clusters = 5  # Number of clusters for segmentation (modify with your desired value)
    image_cv = cv.cvtColor(np.array(placeholder_image), cv.COLOR_RGB2BGR)
    image_cv = cv.GaussianBlur(image_cv, (7, 7), 0)
    pixels = image_cv.reshape(-1, 3).astype(np.float32)

    # Perform K-means clustering
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv.kmeans(pixels, num_clusters, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)

    # Convert the centers to unsigned 8-bit values
    centers = np.uint8(centers)

    # Map the labels to the centers
    segmented_image = centers[labels.flatten()]
    segmented_image = segmented_image.reshape(image_cv.shape)

    # Convert image back to PIL format and display
    segmented_image = Image.fromarray(cv.cvtColor(segmented_image, cv.COLOR_BGR2RGB))
    segmented_image = segmented_image.resize((200, 200))
    segmented_photo = ImageTk.PhotoImage(segmented_image)
    image_label.configure(image=segmented_photo)
    image_label.image = segmented_photo


def co_anh():
    # Convert the image to OpenCV format
    image_cv = cv.cvtColor(np.array(placeholder_image), cv.COLOR_RGB2BGR)
    
    # Create a structuring element for erosion
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (5, 5))
    
    # Apply erosion to the image
    eroded_image = cv.erode(image_cv, kernel, iterations=1)
    
    # Convert the image back to PIL format
    eroded_image = Image.fromarray(cv.cvtColor(eroded_image, cv.COLOR_BGR2RGB))
    eroded_image = eroded_image.resize((200, 200))
    eroded_photo = ImageTk.PhotoImage(eroded_image)
    
    # Update the image_label
    image_label.configure(image=eroded_photo)
    image_label.image = eroded_photo

def enhance_image():
    # Convert the image to the YCrCb color space
    ycrcb = cv.cvtColor(np.array(placeholder_image), cv.COLOR_RGB2YCrCb)
    
    # Extract the Y channel (brightness)
    y = ycrcb[:, :, 0]
    
    # Apply histogram equalization to enhance the brightness
    y_eq = cv.equalizeHist(y)
    
    # Assign the enhanced Y channel back to the YCrCb image
    ycrcb[:, :, 0] = y_eq
    
    # Convert the image back to the RGB color space
    enhanced_image = cv.cvtColor(ycrcb, cv.COLOR_YCrCb2RGB)
    
    # Convert the image to PIL format and resize
    enhanced_image = Image.fromarray(enhanced_image)
    enhanced_image = enhanced_image.resize((200, 200))
    
    # Convert the PIL image to PhotoImage
    enhanced_photo = ImageTk.PhotoImage(enhanced_image)
    
    # Update the image_label
    image_label.configure(image=enhanced_photo)
    image_label.image = enhanced_photo


def global_threshold():
    # Convert the image to grayscale
    grayscale_image = cv.cvtColor(np.array(placeholder_image), cv.COLOR_RGB2GRAY)

    # Calculate the global threshold using the Otsu's method
    _, threshold = cv.threshold(grayscale_image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    
    # Create a binary image based on the threshold
    binary_image = Image.fromarray(threshold)
    binary_image = binary_image.resize((200, 200))

    # Convert the binary image to PhotoImage
    binary_photo = ImageTk.PhotoImage(binary_image)
    
    # Update the image_label
    image_label.configure(image=binary_photo)
    image_label.image = binary_photo

def extract_features_HOG():
    # Convert the image to grayscale
    image_gray = placeholder_image.convert('L')
    image_gray = image_gray.resize((200, 200))
    image_np = np.array(image_gray, dtype=np.uint8)

    # Compute HOG features
    hog_features, hog_image = hog(image_np, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)

    # Convert HOG image to compatible format for displaying with Tkinter
    hog_image = (hog_image * 255).astype(np.uint8)
    hog_image = Image.fromarray(hog_image)
    hog_photo = ImageTk.PhotoImage(hog_image)

    # Update the image_label with the HOG image
    image_label.configure(image=hog_photo)
    image_label.image = hog_photo

def extract_features_SIFT():
    # Convert the image to grayscale
    image_gray = placeholder_image.convert('L')

    # Create a SIFT object
    sift = cv.SIFT_create()

    # Detect and compute keypoints and descriptors using SIFT
    keypoints, descriptors = sift.detectAndCompute(np.array(image_gray), None)

    # Draw keypoints on the image
    image_with_keypoints = cv.drawKeypoints(np.array(image_gray), keypoints, None)

    # Convert the image to PIL format
    image_with_keypoints_pil = Image.fromarray(image_with_keypoints)

    # Resize the image
    image_with_keypoints_pil = image_with_keypoints_pil.resize((200, 200))

    # Convert the image to PhotoImage format for displaying with Tkinter
    image_with_keypoints_photo = ImageTk.PhotoImage(image_with_keypoints_pil)

    # Update the image_label with the image containing keypoints
    image_label.configure(image=image_with_keypoints_photo)
    image_label.image = image_with_keypoints_photo

def extract_edges():
    # Convert the placeholder_image to OpenCV format
    image_cv = cv.cvtColor(np.array(placeholder_image), cv.COLOR_RGB2BGR)

    # Convert the image to grayscale
    gray = cv.cvtColor(image_cv, cv.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv.GaussianBlur(gray, (5, 5), 0)

    # Perform Canny edge detection
    edges = cv.Canny(blurred, 30, 150)

    # Convert the edges image to PIL format and resize
    edges_image = Image.fromarray(edges)
    edges_image = edges_image.resize((200, 200))

    # Convert the PIL image to PhotoImage
    edges_photo = ImageTk.PhotoImage(edges_image)

    # Update the image_label
    image_label.configure(image=edges_photo)
    image_label.image = edges_photo


def load_images_from_folder(folder):
    global image_list
    image_list = []
    for root, dirs, files in os.walk(folder):
        for filename in files:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(root, filename)
                image = Image.open(image_path)
                image = image.resize((200, 200))
                image_list.append((image_path, image))
    return image_list


def show_image(index):
    global current_image_index
    current_image_index = index
    image_path, image = image_list[index]
    photo = ImageTk.PhotoImage(image)
    image_label.configure(image=photo)
    image_label.image = photo


def button_clicked_fd():
    os.makedirs(blur_folder, exist_ok=True)
    for image_path, image in image_list:
        filename = os.path.basename(image_path)
        output_image_path = os.path.join(blur_folder, f"blurred_{filename}")
        blurred_image = image.filter(ImageFilter.BLUR)
        blurred_image.save(output_image_path)


def lamsanganh_fd():
    brightness = 10
    contrast = 2.3
    os.makedirs(lamsang_folder, exist_ok=True)
    for image_path, image in image_list:
        filename = os.path.basename(image_path)
        output_image_path = os.path.join(lamsang_folder, f"adjusted_{filename}")
        adjusted_image = image.point(lambda p: p * contrast + brightness)
        adjusted_image = adjusted_image.resize((200, 200))
        adjusted_image.save(output_image_path)


def lamsacnet_fd():
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=np.float32)
    os.makedirs(lamnet_folder, exist_ok=True)
    for image_path, image in image_list:
        filename = os.path.basename(image_path)
        output_image_path = os.path.join(lamnet_folder, f"sharpened_{filename}")
        image_cv = cv.cvtColor(np.array(image), cv.COLOR_RGB2BGR)
        sharpened_image = cv.filter2D(image_cv, -1, kernel)
        sharpened_image = Image.fromarray(cv.cvtColor(sharpened_image, cv.COLOR_BGR2RGB))
        sharpened_image = sharpened_image.resize((200, 200))
        sharpened_image.save(output_image_path)


def gian_anh_fd():
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (5, 5))
    os.makedirs(giananh_folder, exist_ok=True)
    for image_path, image in image_list:
        filename = os.path.basename(image_path)
        output_image_path = os.path.join(giananh_folder, f"dilated_{filename}")
        image_cv = cv.cvtColor(np.array(image), cv.COLOR_RGB2BGR)
        dilated_image = cv.dilate(image_cv, kernel, iterations=1)
        dilated_image = Image.fromarray(cv.cvtColor(dilated_image, cv.COLOR_BGR2RGB))
        dilated_image = dilated_image.resize((200, 200))
        dilated_image.save(output_image_path)


def co_anh_fd():
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (5, 5))
    os.makedirs(coanh_folder, exist_ok=True)
    for image_path, image in image_list:
        filename = os.path.basename(image_path)
        output_image_path = os.path.join(coanh_folder, f"eroded_{filename}")
        image_cv = cv.cvtColor(np.array(image), cv.COLOR_RGB2BGR)
        eroded_image = cv.erode(image_cv, kernel, iterations=1)
        eroded_image = Image.fromarray(cv.cvtColor(eroded_image, cv.COLOR_BGR2RGB))
        eroded_image = eroded_image.resize((200, 200))
        eroded_image.save(output_image_path)


def enhance_image_fd():
    os.makedirs(nangcaocl_folder, exist_ok=True)
    for image_path, image in image_list:
        filename = os.path.basename(image_path)
        output_image_path = os.path.join(nangcaocl_folder, f"enhanced_{filename}")
        
        # Convert the image to the YCrCb color space
        ycrcb = cv.cvtColor(np.array(image), cv.COLOR_RGB2YCrCb)
        
        # Extract the Y channel (brightness)
        y = ycrcb[:, :, 0]
        
        # Apply histogram equalization to enhance the brightness
        y_eq = cv.equalizeHist(y)
        
        # Assign the enhanced Y channel back to the YCrCb image
        ycrcb[:, :, 0] = y_eq
        
        # Convert the image back to the RGB color space
        enhanced_image = cv.cvtColor(ycrcb, cv.COLOR_YCrCb2RGB)
        
        # Convert the image to PIL format and resize
        enhanced_image = Image.fromarray(enhanced_image)
        enhanced_image = enhanced_image.resize((200, 200))
        
        # Save the enhanced image
        enhanced_image.save(output_image_path)



def global_threshold_fd():
    os.makedirs(laynguong_folder, exist_ok=True)
    for image_path, image in image_list:
        filename = os.path.basename(image_path)
        output_image_path = os.path.join(laynguong_folder, f"thresholded_{filename}")
        
        # Convert the image to grayscale
        grayscale_image = cv.cvtColor(np.array(image), cv.COLOR_RGB2GRAY)
    
        # Calculate the global threshold using the Otsu's method
        _, threshold = cv.threshold(grayscale_image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        
        # Create a binary image based on the threshold
        binary_image = Image.fromarray(threshold)
        binary_image = binary_image.resize((200, 200))
        
        # Save the thresholded image
        binary_image.save(output_image_path)


def extract_features_fd_HOG():
    global folder_path
    os.makedirs(ruttrich_folder_hog, exist_ok=True)

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)
            image_gray = image.convert('L')
            image_gray = image_gray.resize((200, 200))
            image_np = np.array(image_gray, dtype=np.uint8)

            # Compute HOG features
            hog_features, hog_image = hog(image_np, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)

            # Convert HOG image to compatible format for displaying with Tkinter
            hog_image = (hog_image * 255).astype(np.uint8)
            hog_image = Image.fromarray(hog_image)
            hog_photo = ImageTk.PhotoImage(hog_image)

            # Save the HOG image
            output_image_path = os.path.join(ruttrich_folder_hog, f"hog_{filename}")
            hog_image.save(output_image_path)

            # Update the image_label with the HOG image
            image_label.configure(image=hog_photo)
            image_label.image = hog_photo
def extract_features_fd_SIFT():
    global folder_path
    os.makedirs(ruttrich_folder_sift, exist_ok=True)

    # Create a SIFT object
    sift = cv.SIFT_create()

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)
            image_gray = image.convert('L')

            # Detect and compute keypoints and descriptors using SIFT
            keypoints, descriptors = sift.detectAndCompute(np.array(image_gray), None)

            # Draw keypoints on the image
            image_with_keypoints = cv.drawKeypoints(np.array(image_gray), keypoints, None)

            # Convert the image to PIL format
            image_with_keypoints_pil = Image.fromarray(image_with_keypoints)

            # Resize the image
            image_with_keypoints_pil = image_with_keypoints_pil.resize((200, 200))

            # Save the image with keypoints
            output_image_path = os.path.join(ruttrich_folder_sift, f"sift_{filename}")
            image_with_keypoints_pil.save(output_image_path)

            # Convert the image to PhotoImage format for displaying with Tkinter
            image_with_keypoints_photo = ImageTk.PhotoImage(image_with_keypoints_pil)

            # Update the image_label with the image containing keypoints
            image_label.configure(image=image_with_keypoints_photo)
            image_label.image = image_with_keypoints_photo


def extract_edges_fd():
    os.makedirs(trichbien_folder, exist_ok=True)
    for image_path, image in image_list:
        filename = os.path.basename(image_path)
        output_image_path = os.path.join(trichbien_folder, f"edges_{filename}")
        
        # Convert the image to OpenCV format
        image_cv = cv.cvtColor(np.array(image), cv.COLOR_RGB2BGR)

        # Convert the image to grayscale
        gray = cv.cvtColor(image_cv, cv.COLOR_BGR2GRAY)

        # Apply Gaussian blur to reduce noise
        blurred = cv.GaussianBlur(gray, (5, 5), 0)

        # Perform Canny edge detection
        edges = cv.Canny(blurred, 30, 150)

        # Convert the edges image to PIL format and resize
        edges_image = Image.fromarray(edges)
        edges_image = edges_image.resize((200, 200))

        # Save the edges image
        edges_image.save(output_image_path)



def image_segmentation_fd():
    os.makedirs(phanvung_folder, exist_ok=True)
    for image_path, image in image_list:
        filename = os.path.basename(image_path)
        output_image_path = os.path.join(phanvung_folder, f"segmented_{filename}")

        # Perform image segmentation
        num_clusters = 5  # Number of clusters for segmentation (modify with your desired value)
        image_cv = cv.cvtColor(np.array(image), cv.COLOR_RGB2BGR)
        image_cv = cv.GaussianBlur(image_cv, (7, 7), 0)
        pixels = image_cv.reshape(-1, 3).astype(np.float32)

        # Perform K-means clustering
        criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        _, labels, centers = cv.kmeans(pixels, num_clusters, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)

        # Convert the centers to unsigned 8-bit values
        centers = np.uint8(centers)

        # Map the labels to the centers
        segmented_image = centers[labels.flatten()]
        segmented_image = segmented_image.reshape(image_cv.shape)

        # Convert image back to PIL format and save
        segmented_image = Image.fromarray(cv.cvtColor(segmented_image, cv.COLOR_BGR2RGB))
        segmented_image = segmented_image.resize((200, 200))
        segmented_image.save(output_image_path)


def select_image():
    global placeholder_image
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        image = Image.open(file_path)
        image = image.resize((200, 200))
        placeholder_image = image
        photo = ImageTk.PhotoImage(image)
        image_label.config(image=photo)
        image_label.image = photo

def select_folder():
    global folder_path
    folder_path = filedialog.askdirectory()
    if folder_path:
        image_list = load_images_from_folder(folder_path)
        if image_list:
            show_image(0)


button_choose = tk.Button(window, text="Choose Image", bg = 'pink', font='50', command=select_image)
button_choose.grid(column=10, row=1)
button_choose.place(x=350, y=150)

button_blur = tk.Button(window, text="Làm mờ ảnh", font='50', command=button_clicked)
button_blur.grid(column=2, row=9)
button_blur.place(x=50, y=300)


button_bright = tk.Button(window, text="Làm sáng ảnh", font='50', command=lamsanganh)
button_bright.place(x=200, y=300)

button_net = tk.Button(window, text="Làm nét ảnh", font='50', command=lamsacnet)
button_net.place(x=350, y=300)


button_gian = tk.Button(window, text="Làm giãn ảnh", font='50', command=gian_anh)
button_gian.place(x=500, y=300)


button_co = tk.Button(window, text="Làm co ảnh", font='50', command=co_anh)
button_co.place(x=700, y=300)

button_nccla = tk.Button(window, text="Nâng cao chất lượng ảnh", font='50', command=enhance_image)
button_nccla.place(x=50, y=370)

button_nccla = tk.Button(window, text="Lấy ngưỡng toàn cục", font='50', command=global_threshold)
button_nccla.place(x=300, y=370)

button_nccla = tk.Button(window, text="Trích đặc trưng HOG", font='50', command=extract_features_HOG)
button_nccla.place(x=530, y=370)

button_nccla = tk.Button(window, text="Trích đặc trưng SIFT", font='50', command=extract_features_SIFT)
button_nccla.place(x=760, y=370)

button_nccla = tk.Button(window, text="Trích biên", font='50', command=extract_edges)
button_nccla.place(x=1000, y=370)



#### folder ###

button_choose_folder = tk.Button(window, text="Choose Folder", bg = 'violet', font='50', command=select_folder)
button_choose_folder.grid(column=10, row=2)
button_choose_folder.place(x=550, y=150)

button_blur_fd = tk.Button(window, text="Làm mờ bộ ảnh", font='50', command=button_clicked_fd)
button_blur_fd.grid(column=2, row=9)
button_blur_fd.place(x=50, y=450)


button_bright_fd = tk.Button(window, text="Làm sáng bộ ảnh", font='50', command=lamsanganh_fd)
button_bright_fd.place(x=200, y=450)

button_net_fd = tk.Button(window, text="Làm nét bộ ảnh", font='50', command=lamsacnet_fd)
button_net_fd.place(x=400, y=450)


button_gian_fd = tk.Button(window, text="Làm giãn bộ ảnh", font='50', command=gian_anh_fd)
button_gian_fd.place(x=600, y=450)


button_co_fd = tk.Button(window, text="Làm co bộ ảnh", font='50', command=co_anh_fd)
button_co_fd.place(x=800, y=450)

button_nccla_fd = tk.Button(window, text="Nâng cao chất lượng bộ ảnh", font='50', command=enhance_image_fd)
button_nccla_fd.place(x=50, y=530)

button_nccla_fd = tk.Button(window, text="Lấy ngưỡng toàn cục bộ ảnh", font='50', command=global_threshold_fd)
button_nccla_fd.place(x=350, y=530)

button_nccla_fd = tk.Button(window, text="Trích đặc trưng bộ ảnh_HOG", font='50', command=extract_features_fd_HOG)
button_nccla_fd.place(x=50, y=600)
button_nccla_fd = tk.Button(window, text="Trích đặc trưng bộ ảnh_SIFT", font='50', command=extract_features_fd_SIFT)
button_nccla_fd.place(x=400, y=600)

button_nccla_fd = tk.Button(window, text="Trích biên bộ ảnh", font='50', command=extract_edges_fd)
button_nccla_fd.place(x=700, y=600)

button_bright = tk.Button(window, text="Phân vùng ảnh", font='50', command=image_segmentation)
button_bright.place(x=700, y=530) 

button_bright = tk.Button(window, text="Phân vùng bộ ảnh", font='50', command=image_segmentation_fd)
button_bright.place(x=930, y=600) 


window.mainloop()
